####
#### Importing Libraries 

In [1]:
import cv2
import numpy as np
from os import listdir
import matplotlib.pyplot as plt

#### Reading an image

In [2]:
IMAGE="dog.jpg"

#### Setting path of Configuration file 

In [3]:
CONFIG='./yolov3.cfg'

#### Setting path of class file 

In [4]:
CLASSES='./yolov3.txt'

#### Setting path of Weight file 

In [5]:
WEIGHTS='./yolov3.weights'

#### Processing part 1 

In [6]:
classes = None
with open(CLASSES, 'r') as f:
     classes = [line.strip() for line in f.readlines()]
        
scale = 0.00392
conf_threshold = 0.5
nms_threshold = 0.4

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

#### Processing part 2

In [7]:
# function to get the output layer names 
# in the architecture
def get_output_layers(net): 
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

#### Processing part 3 

In [8]:
def processImage(image,index):

    
    Height = image.shape[0]
    Width = image.shape[1]
    # read pre-trained model and config file
    net = cv2.dnn.readNet(WEIGHTS, CONFIG)

    # create input blob 
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input blob for the network
    net.setInput(blob)

    # run inference through the network
    # and gather predictions from output layers
    outs = net.forward(get_output_layers(net))

    # initialization
    class_ids = []
    confidences = []
    boxes = []
    # for each detetion from each output layer 
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
            
    # apply non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # go through the detections remaining
    # after nms and draw bounding box
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
    
        draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    # display output image    
    out_image_name = "object detection"+str(index)
    #plt.imshow(image)
    #save output image to disk
    #plt.axis("off")
    #plt.imshow(image)
    #plt.show()
    #cv2.imwrite("out_1.jpg", image)
    
    loaded_images = list()  
    loaded_images.append(image)
    print(loaded_images)
    
    
# open the video file
cap = cv2.VideoCapture(IMAGE)
index = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    processImage(frame,index)
    index = index + 1
plt.imshow(image)
cv2.destroyAllWindows()

[array([[[ 50,  58,  56],
        [ 51,  59,  57],
        [ 53,  61,  59],
        ...,
        [ 31, 102, 119],
        [ 39,  53,  77],
        [ 45,  59,  83]],

       [[ 51,  59,  57],
        [ 51,  59,  57],
        [ 52,  60,  58],
        ...,
        [ 21,  86, 104],
        [ 41,  54,  73],
        [ 46,  59,  78]],

       [[ 51,  59,  56],
        [ 51,  59,  56],
        [ 52,  60,  57],
        ...,
        [ 10,  64,  83],
        [ 47,  58,  67],
        [ 42,  53,  62]],

       ...,

       [[179, 167, 160],
        [179, 167, 160],
        [183, 170, 163],
        ...,
        [ 64,  63,  80],
        [ 36,  39,  52],
        [ 48,  51,  64]],

       [[181, 170, 161],
        [180, 169, 160],
        [176, 165, 156],
        ...,
        [ 61,  61,  77],
        [ 37,  41,  54],
        [ 52,  56,  69]],

       [[177, 166, 157],
        [178, 167, 158],
        [173, 162, 153],
        ...,
        [ 62,  62,  78],
        [ 33,  37,  50],
        [ 35,  39,  52]

AttributeError: 'NoneType' object has no attribute 'shape'